In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w6p1/sample_submission.csv
/kaggle/input/2022-ai-w6p1/pca_test.csv
/kaggle/input/2022-ai-w6p1/pca_train.csv
/kaggle/input/2022-ai-w6p1/train_label.csv


In [2]:
train_x = pd.read_csv('/kaggle/input/2022-ai-w6p1/pca_train.csv')
train_y = pd.read_csv('/kaggle/input/2022-ai-w6p1/train_label.csv')
sample = pd.read_csv('/kaggle/input/2022-ai-w6p1/sample_submission.csv')
test_x = pd.read_csv('/kaggle/input/2022-ai-w6p1/pca_test.csv')

train_x

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-6.227718,-2.070960,0.674094,-0.742135,0.309149,-4.006772,-0.664624,0.533987,0.024594,-0.195856,...,-2.114192,-0.605770,0.491399,-0.166693,1.289197,-0.403863,0.238731,2.166737,-0.106487,1.486114
1,-6.081077,-1.960213,0.846482,-0.932816,-0.050208,-4.082826,-0.449032,0.647842,0.417809,0.073842,...,-1.000466,-1.175968,0.299093,-1.187048,-0.499101,0.738016,0.632842,0.850430,-2.211427,1.083040
2,-6.394956,-2.954897,1.600033,-0.479463,-1.217065,-3.986472,-0.812934,0.472509,1.536917,1.989765,...,-1.703212,0.713673,0.330699,0.036953,-0.204944,0.930597,-0.693151,-0.572443,0.835598,-0.751483
3,-6.681355,-2.055031,1.144864,-0.517872,-0.168057,-3.929032,-0.718259,0.555827,0.096472,0.724261,...,0.434870,1.571784,-1.266089,1.584173,-0.290301,0.068907,-1.148258,1.904273,0.322937,-0.318100
4,-5.716713,-2.163729,0.877930,-0.605268,0.082372,-4.261466,-0.679734,0.739369,0.490033,0.546437,...,0.626112,-0.435030,2.112485,1.101894,1.746205,-0.445612,-1.518138,0.299177,-0.020655,0.234824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27666,-6.047007,-2.072704,0.668214,-0.254932,0.099894,-1.477478,-0.703836,-0.327026,0.504125,0.222820,...,-1.449840,-0.050902,-1.240334,-0.750522,-1.168852,0.048051,0.432534,-0.003478,-0.609472,-0.348196
27667,-6.569392,-1.932956,0.602381,0.253941,-0.341952,4.064471,-1.502157,-2.467482,0.281797,0.812378,...,0.721433,1.197452,-0.700384,-3.483315,-0.371596,0.478422,-1.537939,2.030344,-1.080110,-0.409972
27668,-6.232087,-1.834862,2.097659,-0.566552,-0.288520,-4.034954,0.457435,0.315611,0.981373,0.483152,...,0.662459,-1.293272,0.067110,0.357502,-0.155424,-0.658622,2.016063,-0.359339,0.993638,0.130081
27669,-6.387326,-0.387639,-1.090173,1.499431,0.629389,31.163396,-3.186039,-12.114586,-0.543894,-1.469931,...,-0.838120,0.584897,-1.355164,-1.358166,1.344522,-0.526878,-0.230260,-0.459776,2.507944,-1.379322


In [3]:
train_y = train_y['label']
train_y

0        136
1         90
2        112
3        165
4         73
        ... 
27666      9
27667    132
27668     18
27669     41
27670    114
Name: label, Length: 27671, dtype: int64

In [4]:
import torch
import torch.optim as optim

torch.manual_seed(777)
x_train = torch.FloatTensor(train_x.values)
x_test = torch.FloatTensor(test_x.values)
y_train = torch.LongTensor(train_y.values)
# y_train = y_train.view([-1, 1])


x_train



tensor([[-6.2277, -2.0710,  0.6741,  ...,  2.1667, -0.1065,  1.4861],
        [-6.0811, -1.9602,  0.8465,  ...,  0.8504, -2.2114,  1.0830],
        [-6.3950, -2.9549,  1.6000,  ..., -0.5724,  0.8356, -0.7515],
        ...,
        [-6.2321, -1.8349,  2.0977,  ..., -0.3593,  0.9936,  0.1301],
        [-6.3873, -0.3876, -1.0902,  ..., -0.4598,  2.5079, -1.3793],
        [-6.3171, -2.1599,  1.0853,  ..., -0.8076,  0.2477,  2.3186]])

In [5]:
# # 모델 정의
# # Layer 정의
# linear = torch.nn.Linear(256,1,bias=True) # Fully Connected Layer
# sigmoid = torch.nn.Sigmoid()

In [6]:
y_train

tensor([136,  90, 112,  ...,  18,  41, 114])

In [7]:
nb_class = 198
nb_data = len(y_train)

y_one_hot = torch.zeros(nb_data, nb_class)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

print(y_one_hot)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [8]:
c = set(train_y)
len(c)

198

In [9]:
## 모델 설계
layer1 = torch.nn.Linear(256,256,bias=True)
layer2 = torch.nn.Linear(256,256,bias=True)
layer3 = torch.nn.Linear(256,256,bias=True)
layer4 = torch.nn.Linear(256,256,bias=True)
layer5 = torch.nn.Linear(256,256,bias=True)
layer6 = torch.nn.Linear(256,198,bias=True)
layer7 = torch.nn.Linear(256,198,bias=True)

sigmoid = torch.nn.Sigmoid()

model = torch.nn.Sequential(layer6,sigmoid)

In [10]:
# 모델 학습 환경 설정
# Binary Cross Entropy Loss
loss = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1)

In [11]:
# for stop in range(10000):
    
#     # 그래디언트 초기화
#     optimizer.zero_grad()
#     # Forward 계산
#     hypothesis = model(x_train)
    
#     # Error 계산
#     cost = loss(hypothesis, y_train)
#     # Backward 계산 
#     cost.backward()
#     # 가중치 갱신
#     optimizer.step()

#     if stop % 100 == 0:
#         print(stop, cost.item())
        
        
for stop in range(10000):
    
    # 그래디언트 초기화
    optimizer.zero_grad()
    # Forward 계산
    hypothesis = model(x_train)
    # Error 계산
    cost = loss(hypothesis, y_one_hot)
#     print(hypothesis)
    # Backward 계산 
    cost.backward()
    # 가중치 갱신
    optimizer.step()

    if stop % 100 == 0:
        print(stop, cost.item())



0 0.9479326605796814
100 0.6660077571868896
200 0.5637044906616211
300 0.4877912998199463
400 0.42982789874076843
500 0.3844655156135559
600 0.3481377065181732
700 0.31876087188720703
800 0.294488787651062
900 0.27429863810539246
1000 0.2571975886821747
1100 0.24263326823711395
1200 0.23005825281143188
1300 0.21908500790596008
1400 0.20951831340789795
1500 0.2009759545326233
1600 0.19344930350780487
1700 0.1867380142211914
1800 0.18070809543132782
1900 0.17526328563690186
2000 0.1703251451253891
2100 0.16582785546779633
2200 0.1617165207862854
2300 0.1579294055700302
2400 0.154458150267601
2500 0.15120182931423187
2600 0.14821800589561462
2700 0.1454489529132843
2800 0.14288662374019623
2900 0.14048129320144653
3000 0.13816654682159424
3100 0.13597536087036133
3200 0.13365642726421356
3300 0.13180439174175262
3400 0.13006223738193512
3500 0.12841914594173431
3600 0.1268671303987503
3700 0.12539903819561005
3800 0.12400835007429123
3900 0.12268918752670288
4000 0.121436208486557
4100 0.

In [12]:
hypothesis = model(x_train)
predict = torch.argmax(hypothesis, dim=1)
print(hypothesis)
print(predict)

accuracy = (predict == y_train).float().mean()
print(accuracy.item())

tensor([[0.0204, 0.0198, 0.0090,  ..., 0.0211, 0.0227, 0.0117],
        [0.0227, 0.0203, 0.0183,  ..., 0.0227, 0.0210, 0.0172],
        [0.0272, 0.0221, 0.0152,  ..., 0.0214, 0.0203, 0.0118],
        ...,
        [0.0198, 0.0206, 0.0190,  ..., 0.0187, 0.0189, 0.0163],
        [0.0192, 0.0229, 0.0349,  ..., 0.0290, 0.0319, 0.0250],
        [0.0206, 0.0199, 0.0121,  ..., 0.0206, 0.0186, 0.0126]],
       grad_fn=<SigmoidBackward0>)
tensor([126,  83, 112,  ...,  18,  41, 114])
0.622059166431427


In [13]:
hypothesis = model(x_test)
predict = torch.argmax(hypothesis, dim=1)
print(hypothesis)
print(predict)

tensor([[0.0225, 0.0180, 0.0130,  ..., 0.0204, 0.0210, 0.0129],
        [0.0247, 0.0205, 0.0114,  ..., 0.0209, 0.0202, 0.0125],
        [0.0222, 0.0211, 0.0142,  ..., 0.0191, 0.0195, 0.0145],
        ...,
        [0.0206, 0.0241, 0.0121,  ..., 0.0197, 0.0193, 0.0091],
        [0.0187, 0.0181, 0.0148,  ..., 0.0190, 0.0188, 0.0136],
        [0.0186, 0.0199, 0.0129,  ..., 0.0191, 0.0195, 0.0119]],
       grad_fn=<SigmoidBackward0>)
tensor([113, 128,  68,  ...,  23,  68, 117])


In [14]:
sample['label'] = predict.detach().numpy()

sample

,id,label
0,0,113
1,1,128
2,2,68
3,3,28
4,4,121
...,...,...
13624,13624,83
13625,13625,69
13626,13626,23
13627,13627,68


In [15]:
sample.to_csv('submit.csv', index = None)